$$ \text{COMPARISON OF A MULTIPLE TIKTOK PROFILES USING WEB-SCRAPED DATA} $$

In this notebook i will be using the search class i have created in fetch_profiles.py to collect a number of user names related by a similar interest. For example fitness. 

In [23]:
from fetch_profiles import *
from profile_scrape import *
import numpy as np
import pandas as pd
from Functions.expected_interactions import *

In [24]:
fitness_related_users = Search("sport",10)
print(fitness_related_users.usernames)
print(len(fitness_related_users.usernames))


['demi.bars', 'jamesbrosnan_longdrive', 'petermannionjr', 'thefoldinglady', 'worldrugby', 'louispickleball', 'daveboxingcoach', 'izzywildgoose', 'jeremylynch']
9


As you can see we have collected 20 users, we can now use the profile anaylis function for all of these. The run time will be very large for this.

In [27]:
profile_objects = []

for i in fitness_related_users.usernames:
    profile_page = Profile(i,10)
    profile_page.get_information()
    profile_page.quit()
    profile_objects.append(profile_page)

Now that we have collected all the data let's have a look at it.

In [28]:
from h11 import Data


usernames = []
followers = []
expected_likes = []
expected_comments = []
expected_shares = []

for i in profile_objects:
    usernames.append(i.username)
    followers.append(i.followers)
    expected_like = expected_interactions(np.array(i.likes))
    expected_likes.append(expected_like)
    expected_comment = expected_interactions(np.array(i.comments))
    expected_comments.append(expected_comment)
    expected_share = expected_interactions(np.array(i.shares))
    expected_shares.append(expected_share)


dataframe = pd.DataFrame({"Username":usernames,"Followers":followers,"Expected Likes":expected_likes,"Expected Comments":expected_comments,"Expected Shares":expected_shares})
print(dataframe)

                 Username   Followers  Expected Likes  Expected Comments  \
0               demi.bars    248100.0     1065.777778          10.400000   
1  jamesbrosnan_longdrive     19400.0       93.666667           4.777778   
2          petermannionjr     64600.0      117.444444           3.111111   
3          thefoldinglady   4300000.0     4744.555556          28.111111   
4              worldrugby   1400000.0     1458.777778           5.222222   
5         louispickleball      1299.0     2585.444444           8.888889   
6         daveboxingcoach     35200.0       55.000000           0.555556   
7           izzywildgoose     65700.0      242.000000           2.625000   
8             jeremylynch  21300000.0   174918.444444        1434.444444   

   Expected Shares  
0         4.111111  
1         2.111111  
2         0.222222  
3        33.333333  
4         4.900000  
5       313.888889  
6         0.000000  
7         2.125000  
8       890.600000  


Using the database above although one can compare the profiles rather well it would be easier to compare a single number. The formular below is a simple way of accounting for all the parameters above.

$$ \text{interaction score} = \frac{{100}{(l + c^2 + s^3)}}{F}   $$

l is expected likes, c is expected comments, s is expected shares. 

In [36]:
interaction_scores = []
for i in range(0,len(usernames)):

    interaction_score = 100*(expected_likes[i]+(expected_comments[i])*2+(expected_shares[i])*2) / followers[i]

    interaction_scores.append("{:.3f}".format(interaction_score))

interaction_score_data_frame = pd.DataFrame({"Username":usernames,"Followers":followers,"Interaction Score":interaction_scores})
print(interaction_score_data_frame)

                 Username   Followers Interaction Score
0               demi.bars    248100.0             0.441
1  jamesbrosnan_longdrive     19400.0             0.554
2          petermannionjr     64600.0             0.192
3          thefoldinglady   4300000.0             0.113
4              worldrugby   1400000.0             0.106
5         louispickleball      1299.0           248.730
6         daveboxingcoach     35200.0             0.159
7           izzywildgoose     65700.0             0.383
8             jeremylynch  21300000.0             0.843


Now we have a database that is much easier to look for good influences to possibly collaborate with. As you can see, for example louispickleball has a value  weirdly high. 

In [52]:
print(profile_objects[5].likes)
likes = np.array(profile_objects[5].likes)
print(likes.mean())
print(likes.mean() + likes.std()*2)

[15.0, 9.0, 102.0, 22900.0, 95700.0, 39.0, 54.0, 17.0, 32.0, 101.0]
11896.9
28753.3726107043
69403.6452214086


This shows the limitations of our current expected interaction function along with the amount of data we have collected. 

Regardless of this anomaly, a sport company can still find this data useful. we can easily see that the top 4 influencers to collaborate with here would be 0(demi.bars), 1(jamesbrosnan_longdrive), 7(izzywildgoose) 8(jeremylynch). Running this program for hundreds of accounts and comparing followers and our 'interaction score' could provide very useful information. Can be done for any catorgary too. 